In [597]:
import numpy as np
import pandas as pd
import string
import os
from collections import Counter
import copy

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
pd.options.display.max_colwidth = 50
pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 500)
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

height has been deprecated.



# Creating base

In [598]:
new_test=pd.read_csv('..//bases/new_test_variants.csv')
new_test_texts = pd.read_csv('..//bases/new_test_text.csv', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"], encoding = "utf-8")
new_test_final=pd.merge(new_test,new_test_texts,how="left",on="ID")

In [599]:
leaks=pd.read_csv('..//bases/s1_add_train.csv')
leaks_1=pd.DataFrame([leaks["ID"],leaks.drop("ID",axis=1).idxmax(axis=1).map(lambda x: x.lstrip('class'))])
leaks_2=leaks_1.T
leaks_2.columns=["ID","Class"]

In [600]:
train = pd.read_csv('..//bases/training_variants')
test = pd.read_csv('..//bases/test_variants')

In [601]:
train_texts = pd.read_csv('..//bases/training_text', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"], encoding = "utf-8")
test_texts = pd.read_csv('..//bases/test_text', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"], encoding = "utf-8")

In [602]:
train = pd.merge(train, train_texts, how='left', on='ID')
test = pd.merge(test, test_texts, how='left', on='ID')

In [603]:
leaks_3=pd.merge(leaks_2,test[test.ID.isin(leaks_2.ID)])
leaks_final=pd.merge(leaks_3,test_texts[test_texts.ID.isin(leaks_3.ID)])

In [604]:
train_all = pd.concat([train,leaks_final]) #adding first stage

In [605]:
merge_match = new_test.merge(train_all, left_on=['Gene', 'Variation'], right_on = ['Gene', 'Variation'])
Index_leak = merge_match.ID_x - 1
new_test_index = [item for item in new_test_final.index if item not in list(Index_leak)]
test_no_leaks = new_test_final.iloc[new_test_index]
test_no_leaks

,ID,Gene,Variation,Text
2,3,WNT4,E216G,Mycosis fungoides and Sézary syndrome are prim...
3,4,SUCLA2,G118R,Regulated progression through the cell cycle ...
5,6,CHEK2,E239K,The nuclei that laboratories solution p53 KIT ...
6,7,CHST3,T141M,Myeloid differentiation 88 (MyD88) is the key ...
7,8,RNF6,G244D,Human ESCCs 2 occur frequently worldwide (1) ....
8,9,SPAST,C448Y,large were of activity growth this product tol...
10,11,SCN4A,V445M,Endometrial carcinoma is the most common gynec...
14,15,ERBB2,G746S,The protein-kinase family is the most frequent...
15,16,TP53,Y234S,Among the best-studied therapeutic targets in ...
16,17,RAB27A,A87P,"Introduction In recent years, a better unders..."


In [665]:
train_all['Substitutions_var'] = train_all.Variation.apply(lambda x: bool(re.search('^[A-Z]\\d+[A-Z*]$', x))*1)
new_train = train_all[train_all['Substitutions_var']==1]

In [607]:
#### process the train and test set together
data_all = pd.concat((new_train, test_no_leaks), axis=0, ignore_index=True)
data_all = data_all[['Class', 'Gene', 'ID', 'Variation', 'Text']] # just reordering
data_all_backup = data_all[:] ## We keep backup in case we need to use again
data_all

,Class,Gene,ID,Variation,Text
0,2,CBL,1,W802*,Abstract Background Non-small cell lung canc...
1,2,CBL,2,Q249E,Abstract Background Non-small cell lung canc...
2,3,CBL,3,N454D,Recent evidence has demonstrated that acquired...
3,4,CBL,4,L399V,Oncogenic mutations in the monomeric Casitas B...
4,4,CBL,5,V391I,Oncogenic mutations in the monomeric Casitas B...
5,5,CBL,6,V430M,Oncogenic mutations in the monomeric Casitas B...
6,4,CBL,8,Y371H,Abstract Juvenile myelomonocytic leukemia (JM...
7,4,CBL,9,C384R,Abstract Juvenile myelomonocytic leukemia (JM...
8,4,CBL,10,P395A,Oncogenic mutations in the monomeric Casitas B...
9,4,CBL,11,K382E,Noonan syndrome is an autosomal dominant conge...


In [608]:
#Transform Amino Acid (AA) Letter to their three-letter abbreviation in order to find them in the text when they appear
One_to_Three_AA = {'C': 'Cys', 'D': 'Asp', 'S': 'Ser', 'Q': 'Gln', 'K': 'Lys',
         'I': 'Ile', 'P': 'Pro', 'T': 'Thr', 'F': 'Phe', 'N': 'Asn', 
         'G': 'Gly', 'H': 'His', 'L': 'Leu', 'R': 'Arg', 'W': 'Trp', 
         'A': 'Ala', 'V': 'Val', 'E': 'Glu', 'Y': 'Tyr', 'M': 'Met'}
pattern = re.compile('|'.join(One_to_Three_AA.keys()))

# Adding extra feature(s)

In [609]:
# Feature for the length of the text
data_all["Text_words"] = data_all["Text"].map(lambda x: len(str(x).split(" ")))

# Substitutions (subs)
## Functions for pre-processing of subs

In [610]:
# find_sub return the substituions that are in text and those that are not
def find_sub(data):    
    Boolean = [data.Variation[i][:-1] in data.Text[i] or #case 1.
               pattern.sub(lambda x: One_to_Three_AA[x.group()], data.Variation[i][:-1]) # case2
               in data.Text[i]  for i in data.index] ## because new indexing we use 
    
    sub_in_text = data[Boolean]
    not_Boolean = [not i for i in Boolean]  
    sub_not_in_text = data[not_Boolean]
    
    return sub_in_text, sub_not_in_text

# find_sub_numberChange searches for other number of a substitution i.e. G12V -> G_V because sometimes mistake in entry
# Is currently without One_to_three substitution or Variation[:-1] only the full variation
def find_sub_numberChange(data):
    Booleans = [] #will contain the different Booleans if found in text
    for i in data.index:
        split_variation = re.split('(\d+)', data.Variation[i]) # split based on a number
        first_Amino = re.escape(split_variation[0]) #re.escpae uses variable as regex
        last_Amino = re.escape(split_variation[-1])
        new_regex  = first_Amino + r"\d+" + last_Amino
        Boolean = bool(re.search(new_regex, data.Text[i]))
        Booleans.append(Boolean)
        
    sub_number_in_text = data[Booleans]
    not_Boolean = [not i for i in Booleans]  
    sub_number_no_text = data[not_Boolean]
    
    return sub_number_in_text, sub_number_no_text

# for substitutions that are still not found, use other keywords
def find_sub_pattern(data, pattern):    
    Boolean = [pattern in data.Text[i] for i in data.index] ## because new indexing we use 
    
    sub_in_text = data[Boolean]
    not_Boolean = [not i for i in Boolean]  
    sub_not_in_text = data[not_Boolean]
    
    return sub_in_text, sub_not_in_text


In [611]:
##### get_sentences_sub use a window to extract sentences where the subs appear. 
# If window_left & window_right = 0 => just taking the sentences with subs

def get_sentences_sub(data, splitted_sentences):
    data.index = range(len(data)) #makes sure that index is right
    sentences_with_sub = [[] for _ in range(len(data))]
    for i in range(len(splitted_sentences)):
        sentences = splitted_sentences[i]
        one_to_three_variation = pattern.sub(lambda x: One_to_Three_AA[x.group()], data.Variation[i][:-1])
        Variation = data.Variation[i][:-1]    
        for j in range(len(sentences)):                              
            if (Variation in sentences[j]) or (one_to_three_variation in sentences[j]):
                new_regex = re.escape(Variation) + r"[\S]*" ###  r"[\S]*" because we look for Variation[:-1] not just Variation
                sentences[j] = re.sub(new_regex, ' placeholderMutation', sentences[j]) #case 1
                ### We add the space to ' placeholderMutation' because sometimes there are letters in front of it
                new_regex = re.escape(one_to_three_variation) + r"[\S]*"
                sentences[j] = re.sub(new_regex, ' placeholderMutation', sentences[j]) #case 2
                sentences_with_sub[i].extend(sentences[j:j+1])
                
    return sentences_with_sub

##### get_sentences_sub_number use a window to extract sentences where the subs appear that have different number i.e. G12V -> G_V

def get_sentences_sub_number(data, splitted_sentences, window_left, window_right):
    #position_sentences = [[] for _ in range(len(data))]  #### currently not used
    data.index = range(len(data))
    sentences_with_sub_number = [[] for _ in range(len(data))]
    for i in range(len(splitted_sentences)):
        sentences = splitted_sentences[i]
        split_variation = re.split('(\d+)', data.Variation[i]) # split based on a number
        first_Amino = re.escape(split_variation[0]) #re.escpae uses variable as regex
        last_Amino = re.escape(split_variation[-1])
        new_regex  = first_Amino + r"\d+" + last_Amino
        
        for j in range(len(sentences)):
            Boolean = bool(re.search(new_regex, sentences[j]))            
            if Boolean:
                sentences[j] = re.sub(new_regex, ' placeholderMutation', sentences[j]) # Again replacing the sentence with placeholder
                sentences_with_sub_number[i].extend(sentences[j:j+1])
    
    return sentences_with_sub_number

# for substitutions that are still not found, use other keywords
def get_sentences_pattern(data, splitted_sentences, pattern, window_left, window_right):
    data.index = range(len(data)) #makes sure that index is right
    sentences_with_sub = [[] for _ in range(len(data))]
    for i in range(len(splitted_sentences)):
        sentences = splitted_sentences[i] 
        for j in range(len(sentences)):                              
            if (pattern in sentences[j]):
                sentences_with_sub[i].extend(sentences[j:j+1])               
    
    return sentences_with_sub

In [612]:
###### We use a window when sentences are too low. For 5 <= LENGTH <= 10: window of 1, for <= 5: window of 2
###### 

def get_window_sub(data, splitted_sentences, lengths):
    data.index = range(len(data)) #makes sure that index is right
    sentences_with_sub = [[] for _ in range(len(data))]
    for i in range(len(splitted_sentences)):
        sentences = splitted_sentences[i]
        length = lengths[i]
        if length < 2:
            window=3
        elif length >= 6:
            window=1
        else:
            window=2
        
        one_to_three_variation = pattern.sub(lambda x: One_to_Three_AA[x.group()], data.Variation[i][:-1])
        Variation = data.Variation[i][:-1] 
        all_sentences = []
        for j in range(len(sentences)):
            if (Variation in sentences[j]) or (one_to_three_variation in sentences[j]):
                new_regex = re.escape(Variation) + r"[\S]*" ###  r"[\S]*" because we look for Variation[:-1] not just Variation
                other_regex = re.escape(one_to_three_variation) + r"[\S]*"
                for sentence in sentences[max(j-window,0) : min(j+1+window, len(sentences)-1)]:
                    sentence = re.sub(new_regex, ' placeholderMutation', sentence) #case 1
                    sentence = re.sub(other_regex, ' placeholderMutation', sentence) #case 2 after case 1
                    all_sentences.append(sentence)
        sentences_with_sub[i] = all_sentences
                    
    return sentences_with_sub

def get_window_sub_number(data, splitted_sentences, lengths):
    #position_sentences = [[] for _ in range(len(data))]  #### currently not used
    data.index = range(len(data))
    sentences_with_sub_number = [[] for _ in range(len(data))]
    for i in range(len(splitted_sentences)):
        sentences = splitted_sentences[i] 
        length = lengths[i]
        if length < 2:
            window=3
        elif length >= 6:
            window=1
        else:
            window=2
            
        split_variation = re.split('(\d+)', data.Variation[i]) # split based on a number
        first_Amino = re.escape(split_variation[0]) #re.escpae uses variable as regex
        last_Amino = re.escape(split_variation[-1])
        new_regex  = first_Amino + r"\d+" + last_Amino    
        
        all_sentences = []
        for j in range(len(sentences)):
            Boolean = bool(re.search(new_regex, sentences[j]))            
            if Boolean:
                for sentence in sentences[max(j-window,0) : min(j+1+window, len(sentences)-1)]:
                    sentence = re.sub(new_regex, ' placeholderMutation', sentence)
                    all_sentences.append(sentence)
        sentences_with_sub_number[i] = all_sentences
    
    return sentences_with_sub_number



In [613]:
#### Converts list of sentences into one string of sentences for each document => to use for tfidf etc.
def sentences_to_string(sentences_list):
    sentence_strings = []
    for sentences in sentences_list:
        sentence_string =  ' '.join(str(sentence) for sentence in sentences)
        sentence_strings.append(sentence_string)
    
    return sentence_strings 

## Subs processing of the data set 

In [614]:
######### First find those that have the format of being a substitution in data
data_all['Substitutions_var'] = data_all.Variation.apply(lambda x: bool(re.search('^[A-Z]\\d+[A-Z*]$', x))*1) #multiplying by 1 converts True to 1, False to 0 => Maybe modify this later?
data_all['null'] = data_all.Variation.apply(lambda x: bool(re.search('null', x))*1)
data_all = data_all[(data_all['Substitutions_var']==1) | (data_all['null']==1) ] ### Now we know the index of where a substitution occurs - the data_sub
data_all = data_all.loc[:, 'Class':'Text_words']
data_sub = data_all
print("Length of total subs: %i" %len(data_sub)) # other ways to process it like finding the word 'mutation' 
data_all

Length of total subs: 3544


,Class,Gene,ID,Variation,Text,Text_words
0,2,CBL,1,W802*,Abstract Background Non-small cell lung canc...,5783
1,2,CBL,2,Q249E,Abstract Background Non-small cell lung canc...,5783
2,3,CBL,3,N454D,Recent evidence has demonstrated that acquired...,5625
3,4,CBL,4,L399V,Oncogenic mutations in the monomeric Casitas B...,6248
4,4,CBL,5,V391I,Oncogenic mutations in the monomeric Casitas B...,6248
5,5,CBL,6,V430M,Oncogenic mutations in the monomeric Casitas B...,6248
6,4,CBL,8,Y371H,Abstract Juvenile myelomonocytic leukemia (JM...,12009
7,4,CBL,9,C384R,Abstract Juvenile myelomonocytic leukemia (JM...,5761
8,4,CBL,10,P395A,Oncogenic mutations in the monomeric Casitas B...,6248
9,4,CBL,11,K382E,Noonan syndrome is an autosomal dominant conge...,12251


### Subs inside the text

#### Subs inside the text: all sentences

In [615]:
## First consider the subs that appear in text
sub_text, sub_no_text = find_sub(data_sub) 

In [616]:
## use tokenizer to split into sentences of all the subs in text 
NLTK_sub = [sent_tokenize(sub_text.Text[i]) for i in sub_text.index] 

In [617]:
NLTK_window = copy.deepcopy(NLTK_sub) # a deep copy is necessary besauce something magical and strange happening without it
NLTK_copy = copy.deepcopy(NLTK_sub) # again for backup purposes

In [618]:
# extract window for the sub sentences where they appear
# !! Use [:] because it makes a copy and doesn't change anything to the original indexes
sub_sentences = get_sentences_sub(sub_text[:], NLTK_copy[:]) # choosing for window 0 as default now
sub_sentences_string = sentences_to_string(sub_sentences)

In [619]:
# Replace text in data_all
data_all.Text.loc[sub_text.index] = sub_sentences_string
data_all.Text.loc[sub_text.index]

C:\Users\rafiz\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


0       Using select c-CBL somatic mutations such as S...
1       Using select c-CBL somatic mutations such as S...
2       Most of the changes were novel, although 4 cas...
3       Finally, the third group constituted mutations...
4       Finally, the third group constituted mutations...
5       The second group of mutants (M374V,  placehold...
6       We investigated the mechanism by which CBL- pl...
7       We investigated the mechanism by which CBL-Y37...
8       Finally, the third group constituted mutations...
9       Purified PCR amplicons (CBLWT, CBLC381A, CBL p...
10      Purified PCR amplicons (CBLWT, CBLC381A, CBLK3...
11      Purified PCR amplicons (CBLWT, CBL placeholder...
12      The second group of mutants (M374V, V430M,  pl...
13      Purified PCR amplicons (CBLWT, CBLC381A, CBLK3...
14      When introduced into Lin- Sca1+ c-Kit+ (LSK) H...
15      The second group of mutants ( placeholderMutat...
16      When introduced into Lin- Sca1+ c-Kit+ (LSK) H...
17      Using 

#### Subs inside the text: sentences with length <=10

In [620]:
######### We rerun for window. Length<2 : w = 3, Length >=6: w = 1, Length in between: w = 2
indexes = [index for index, sentences in enumerate(sub_sentences) if len(sentences) <= 10]
sentence_lengths = [len(sentences) for index, sentences in enumerate(sub_sentences) if len(sentences) <= 10]
NLTK_sub_window = [NLTK_window[i] for i in indexes]

In [621]:
new_index = sub_text.index[indexes]
sub_window = sub_text.loc[new_index] # gets the subs with the low length
sub_window #1524 cases 

,Class,Gene,ID,Variation,Text,Text_words
2,3,CBL,3,N454D,Recent evidence has demonstrated that acquired...,5625
3,4,CBL,4,L399V,Oncogenic mutations in the monomeric Casitas B...,6248
4,4,CBL,5,V391I,Oncogenic mutations in the monomeric Casitas B...,6248
5,5,CBL,6,V430M,Oncogenic mutations in the monomeric Casitas B...,6248
8,4,CBL,10,P395A,Oncogenic mutations in the monomeric Casitas B...,6248
12,5,CBL,14,P428L,Oncogenic mutations in the monomeric Casitas B...,6248
14,4,CBL,17,Q367P,Acquired uniparental disomy (aUPD) is a common...,3013
15,5,CBL,18,M374V,Oncogenic mutations in the monomeric Casitas B...,6248
16,4,CBL,19,Y371S,Acquired uniparental disomy (aUPD) is a common...,3013
18,4,CBL,21,C396R,Oncogenic mutations in the monomeric Casitas B...,6248


In [622]:
sub_window_sentences = get_window_sub(sub_window[:], NLTK_sub_window[:], sentence_lengths) 
sub_window_sentences = [sorted(set(sentences), key = sentences.index) for sentences in sub_window_sentences] # removes duplicates
sub_window_string = sentences_to_string(sub_window_sentences)

In [623]:
# Finally: replacing those texts in data_all
data_all.Text.loc[new_index] = sub_window_string
data_all.Text.loc[new_index]

C:\Users\rafiz\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


2       HRM analysis of CBL exons 8 and 9 in blast cri...
3       Only V430M mutant was on the borderline with t...
4       Only V430M mutant was on the borderline with t...
5       Figure 4 Figure 4 Structures of wild type (gre...
8       Only V430M mutant was on the borderline with t...
12      Finally, the Y371H mutation not only abolished...
14      Transformed NIH3T3 cells showed PI3 kinase-dep...
15      Figure 4 Figure 4 Structures of wild type (gre...
16      Transformed NIH3T3 cells showed PI3 kinase-dep...
18      HEK293T cells were transfected with plasmids e...
19      Only V430M mutant was on the borderline with t...
20      Overall survival (A) and progression-free surv...
21      We fully sequenced the coding exons of these g...
22      Overall survival (A) and progression-free surv...
24      Both families contain compound heterozygotes. ...
26      Here we describe 2 families in which 2 TERT mu...
27      Both families contain compound heterozygotes. ...
28      Both f

### Subs with a different number inside the text

#### Subs with a different number inside the text: all sentences


In [624]:
# now the subs that don't appear in text: one reason is different number in the substitution. f.e. G12V -> G13V 
sub_number_text, sub_number_no_text = find_sub_numberChange(sub_no_text) # 131 cases with number change

In [625]:
## use tokenizer to split into sentences of subs that have different number in text 
NLTK_sub_number = [sent_tokenize(sub_number_text.Text[i]) for i in sub_number_text.index]
NLTK_sub_number_window = copy.deepcopy(NLTK_sub_number) # a deep copy is necessary besauce something magical and strange happening without it
NLTK_sub_number_copy = copy.deepcopy(NLTK_sub_number) # again for backup purposes

In [626]:
# extract window for the sub sentences where they appear
# !! Use [:] because it makes a copy and doesn't change anything to the original indexes
sub_number_sentences = get_sentences_sub_number(sub_number_text[:], NLTK_sub_number_copy[:], window_left = 0, window_right = 0)
sub_number_string = sentences_to_string(sub_number_sentences)

In [627]:
data_all.Text.loc[sub_number_text.index] = sub_number_string #iloc for indexing based on integers
data_all.Text.loc[sub_number_text.index]

C:\Users\rafiz\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


37      His-tagged versions of the catalytic region of...
41      His-tagged versions of the catalytic region of...
53      His-tagged versions of the catalytic region of...
55      His-tagged versions of the catalytic region of...
58      His-tagged versions of the catalytic region of...
62      While both were missense mutations, in silico ...
150      placeholderMutation and T847I were also found...
153     Complex mutation  placeholderMutation and dele...
191     As a positive control, we introduced the well-...
198     And in the series of Pallis and colleagues (12...
211     Recurrent mutations in H3F3A, which encodes th...
212     Histone H3-K42me2 may regulate transcription  ...
213     Recurrent mutations in H3F3A, which encodes th...
394     Both combinations contained mutations of codon...
474     In the present study, we have focused on two d...
494     Such mutant amino acid specificity was also ap...
578     In contrast, ectopic expression of FGFR4  plac...
823     Recent

#### Subs with a different number inside the text: sentences with length <= 10

In [628]:
######### ######### We rerun for window. Length<2 : w = 3, Length >=6: w = 1, Length in between: w = 2
indexes = [index for index, sentences in enumerate(sub_number_sentences) if len(sentences) <= 10]
sentence_lengths = [len(sentences) for index, sentences in enumerate(sub_number_sentences) if len(sentences) <= 10]
NLTK_window = [NLTK_sub_number[i] for i in indexes]

In [629]:
new_index = sub_number_text.index[indexes]
sub_number_window = sub_number_text.loc[new_index] # gets the subs with the low length
sub_number_window #87 cases 

,Class,Gene,ID,Variation,Text,Text_words
37,4,PTPRT,44,T1365M,"Tyrosine phosphorylation, regulated by protein...",1958
41,4,PTPRT,49,R1343L,"Tyrosine phosphorylation, regulated by protein...",1958
53,4,PTPRT,61,R1209W,"Tyrosine phosphorylation, regulated by protein...",1958
150,2,EGFR,208,V774A,Purpose: Clinical features of epidermal growth...,4691
153,2,EGFR,213,V774M,Purpose: Clinical features of epidermal growth...,4691
191,7,EGFR,267,G810S,Purpose: Epidermal growth factor receptor (EGF...,4201
198,2,EGFR,275,L838P,Purpose: Clinical features of epidermal growth...,4691
213,8,H3F3A,308,G35V,The DNA entry and exit points on the nucleosom...,17067
394,4,TP53,521,E286K,The transcription factor and tumor suppressor ...,4177
494,1,FBXW7,632,G423R,Background Melanoma is a heterogeneous tumor ...,10992


In [630]:
sub_window_sentences = get_window_sub_number(sub_number_window[:], NLTK_window[:], sentence_lengths) 
sub_window_sentences = [sorted(set(sentences), key = sentences.index) for sentences in sub_window_sentences] # removes duplicates
sub_window_string = sentences_to_string(sub_window_sentences)
sub_window_sentences[0]

['1) would result in polypeptides devoid of C-terminal phosphatase catalytic domains, thereby leading to inactivation of the phosphatase.',
 'To evaluate whether tumor-specific point mutations alter phosphatase activity, we biochemically tested mutant versions of PTPRT, the most frequently mutated PTP in the superfamily.',
 'Mutations in both intracellular PTP domains (D1 and D2) were evaluated.',
 'His-tagged versions of the catalytic region of wild-type PTPRT, two D1 mutants (Q987K and N1128I), and three D2 mutants (R1212W, R1346L, and  placeholderMutation) were produced in bacteria and analyzed for phosphatase activity by using 6,8-difluoro-4-methylumbelliferyl phosphate (DiFMUP) as a substrate (Fig.',
 '2) (12).',
 'All D1 and D2 mutants had reduced phosphatase activity compared with the wild-type protein (Fig.',
 '2).']

In [631]:
data_all.Text.loc[new_index] = sub_window_string
data_all.Text.loc[new_index]

C:\Users\rafiz\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


37      1) would result in polypeptides devoid of C-te...
41      1) would result in polypeptides devoid of C-te...
53      1) would result in polypeptides devoid of C-te...
150     Identifications of very rare EGFR mutations ma...
153     By searching the database (38), we found that ...
191     To test more directly whether the mutations EG...
198     In the large series of Shigematsu and colleagu...
213     Somatic mutations in the H3.3-ATRX-DAXX chroma...
394     Table 1. Intragenic suppressor mutations obtai...
494     Only colorectal tumors had KRAS A146 mutations...
578     None of the somatic alleles of EPHA3, ERBB4, F...
825     EZH2 is the catalytic subunit of the PRC2 comp...
827     EZH2 is the catalytic subunit of the PRC2 comp...
830     Fifteen of 16 mutations in ERCC2, a nucleotide...
831     Fifteen of 16 mutations in ERCC2, a nucleotide...
862     Summary During the past decade, the treatment ...
865     The H1112L and H1112Y mutations were not detec...
869     Mutati

In [632]:
data_all.loc[len(new_train):].Text[data_all.loc[len(new_train):].index[4]]

'We found three mutations in three ESCC primary tumors (Fig. 2 ⇓ ; Table 2 ⇓ ) and one mutation in a tumor cell line (Table 2) ⇓ . The amino acid changes in the mutations were R102K, A242T,  placeholderMutation and S623N. We also analyzed the blood DNAs from the three patients with mutations in the RNF6 gene, and all contained the wild-type allele (Fig. 2) ⇓ , thus indicating that the mutations in the ESCC primary tumors were acquired during tumorigenesis in somatic cells. RNF6 contains three domains: an arginine domain (amino acids 292–424), a poly-aspartic acid domain (amino acids 598–601), and a Zinc finger domain (amino acids 632–673). The mutation S623N, which is located near the Zinc finger domain, may affect its DNA-binding activity. Both A242T and  placeholderMutation may perturb the structure and function of the RNF6 protein. In the first case, the side chain of amino acid Thr is larger than the amino acid Ala. In the second case, the amino acid substitution gains a negatively

#### Subs that are still not found

In [633]:
sub_number_no_text.shape # 365 left

(365, 6)

In [634]:
sub_number_no_text.loc[len(new_train):].shape # 62 in 

(62, 6)

In [635]:
sub_number_no_text.loc[len(new_train):] # A lot of sentences starting with 'Among...'

,Class,Gene,ID,Variation,Text,Text_words
2933,NaN,TP53,16,Y234S,Among the best-studied therapeutic targets in ...,15936
2937,NaN,CSF1R,20,Y969H,The FMS gene encodes the functional cell surfa...,8456
2944,NaN,SYT6,31,A406T,The phosphatidylinositol-3-kinase (PI3K)/serin...,2588
2949,NaN,CSF1R,38,Y969F,Clinical characteristics.Adult-onset leukoence...,17645
2962,NaN,STK11,58,G163C,Germline mutation in serine/threonine kinase 1...,2767
2979,NaN,DCC,87,P1375H,DCC is a candidate tumor-suppressor gene encod...,204
2990,NaN,ACVR1,104,R258M,Among the best-studied therapeutic targets in ...,4479
3018,NaN,STK11,144,R86G,Germline mutation in serine/threonine kinase 1...,2767
3026,NaN,AKT2,156,S302G,Ongoing cancer genome characterization studies...,6526
3030,NaN,MYC,161,P58L,Among the best-studied therapeutic targets in ...,4479


In [664]:
len(sub_number_no_text.loc[len(new_train):][sub_number_no_text.loc[len(new_train):]['Text_words']==4479])

0

In [637]:
# subs that are still not found, look based on a pattern like the word 'mutat' in text
sub_pattern_text, sub_pattern_no_text = find_sub_pattern(sub_number_no_text, 'mutat') # 

In [638]:
sub_pattern_no_text # only 14 texts that still don't include sentences with the word 'mutat', of which three nulls and low text_words length

,Class,Gene,ID,Variation,Text,Text_words
108,5,EGFR,140,I491M,The accurate determination of perfluoroalkyl s...,430
111,2,EGFR,145,K467T,The accurate determination of perfluoroalkyl s...,430
185,2,EGFR,259,S464L,The accurate determination of perfluoroalkyl s...,430
292,4,TP53,416,P151S,The effects of chlorpromazine on various prope...,718
841,1,FANCA,1109,S1088F,null,1
1078,6,FGFR3,1407,K508M,null,1
1255,4,VHL,1613,L158Q,The case of a 40-year-old woman with severe ed...,53
1524,7,CARD11,1936,G116S,Left ventricular (LV) remodeling is a signific...,3681
1528,7,CARD11,1942,E127G,Left ventricular (LV) remodeling is a signific...,3681
1820,7,JAK1,2302,R724H,Regulatory T (T reg) cells have a major role i...,8287


In [639]:
## use tokenizer to split into sentences of subs that have keyword 'mutat'
NLTK_sub_pattern = [sent_tokenize(sub_pattern_text.Text[i]) for i in sub_pattern_text.index]

In [640]:
# !! Use [:] because it makes a copy and doesn't change anything to the original indexes
sub_pattern_sentences = get_sentences_pattern(sub_pattern_text[:], NLTK_sub_pattern[:], 'mutat', window_left = 0, window_right = 0)
sub_pattern_string = sentences_to_string(sub_pattern_sentences)

In [641]:
data_all.Text.iloc[sub_pattern_text.index] = sub_pattern_string #iloc for indexing based on integers
data_all.Text.iloc[sub_pattern_text.index]

C:\Users\rafiz\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


39      Mutations in CBL, encoding an E3 ubiquitin lig...
67      This D1 domain contains only two serine residu...
83      Identification of a high-risk disease-causing ...
85      In this study, we profiled 70 colorectal cance...
87      Identification of a high-risk disease-causing ...
88      Most Lynch Syndrome-associated CRCs are report...
90      Identification of a high-risk disease-causing ...
91      Germ-line MSH6 mutations, which are rare in HN...
107     In 2011, whole-exome sequencing studies showed...
113     We investigated the association between mutati...
116     Clinical data and epidermal growth factor rece...
122     Clinical data and epidermal growth factor rece...
126     Currently available methods of EGFR mutation d...
131     Those were analyzed for EGFR and k-ras (Kirste...
135     Purpose: Clinical reports about responsiveness...
137     ‘Classical' mutations in the EGFR tyrosine kin...
154     Purpose: Clinical features of epidermal growth...
159     Findin

# Extra investigation of test text

In [642]:
test_all = data_all.loc[len(new_train):]
all_text = list(test_all.Text)
test_all

,Class,Gene,ID,Variation,Text,Text_words
2925,NaN,WNT4,3,E216G,This unchanged corresponded consistent express...,8638
2926,NaN,SUCLA2,4,G118R,On placeholderMutation difference first expre...,6221
2927,NaN,CHEK2,6,E239K,"In 1996, two independent groups confirmed that...",3431
2928,NaN,CHST3,7,T141M,We point S12 the the overexpression OSCST pla...,9777
2929,NaN,RNF6,8,G244D,We found three mutations in three ESCC primary...,1757
2930,NaN,SPAST,9,C448Y,Interestingly also indeed of novel and placeh...,3199
2931,NaN,SCN4A,11,V445M,Based of MATERIALS new concentrations placeho...,7118
2932,NaN,ERBB2,15,G746S,To assess our ability to detect all 471 mutati...,11645
2933,NaN,TP53,16,Y234S,Among the best-studied therapeutic targets in ...,15936
2934,NaN,RAB27A,17,A87P,If cases used of 0.8 proof-of-principle with a...,9637


In [643]:
positions_machine = []
all_together = []
for i in range(len(all_text)):
    text = all_text[i]
    if text.count('placeholderMutation') < 11:
        all_together.append((i, text.count('placeholderMutation')))
        positions_machine.append(i)

In [663]:
data_all.loc[len(new_train):] 

,Class,Gene,ID,Variation,Text,Text_words,NOTCH1,MED12,RAD54L,CTNNB1,...,MET,PTEN,MYC,PIK3CA,BRAF,IDH2,XPO1,TP53,RAD50,SMAD4


In [645]:
important_texts = [all_text[i] for i in positions_machine]
test_all.iloc[positions_machine]

,Class,Gene,ID,Variation,Text,Text_words
2929,NaN,RNF6,8,G244D,We found three mutations in three ESCC primary...,1757
2932,NaN,ERBB2,15,G746S,To assess our ability to detect all 471 mutati...,11645
2933,NaN,TP53,16,Y234S,Among the best-studied therapeutic targets in ...,15936
2937,NaN,CSF1R,20,Y969H,This study reports on the frequency of point m...,8456
2944,NaN,SYT6,31,A406T,Each tumor encoded 18 shared and nine private ...,2588
2949,NaN,CSF1R,38,Y969F,Individuals with ALSP usually have an affected...,17645
2952,NaN,SMAD4,43,G386C,We found that gastric cancers were more simila...,4479
2954,NaN,ERBB2,47,V762M,"We repeated the experiments thrice, including ...",17960
2956,NaN,MET,50,M1268T,"Before these results were available, the patie...",9916
2959,NaN,KRAS,54,Q61K,We reported recently that activating BRAF muta...,20483


In [646]:
test_important = test_all.iloc[positions_machine]

In [647]:
len(test_important)

126

# Adding gene dummy

In [648]:
shared_genes = list(set(train_sub.Gene).intersection(set(test_all.Gene))) # the shared genes are those that appear in both
count_important = 0
for gene in test_important.Gene:
    if gene in shared_genes:
        count_important += 1

count_important # so 104 EXAMPLES in test where same gene appears as in the impo

104

In [649]:
shared_genes

['NOTCH1',
 'MED12',
 'RAD54L',
 'CTNNB1',
 'EPAS1',
 'IDH1',
 'CDK4',
 'PTPN11',
 'MAP2K1',
 'CDKN2A',
 'VHL',
 'DNMT3A',
 'FGFR3',
 'FGFR2',
 'EGFR',
 'ERBB2',
 'KRAS',
 'STK11',
 'CREBBP',
 'RAC1',
 'ACVR1',
 'CHEK2',
 'AKT2',
 'NRAS',
 'MET',
 'PTEN',
 'MYC',
 'PIK3CA',
 'BRAF',
 'IDH2',
 'XPO1',
 'TP53',
 'RAD50',
 'SMAD4']

In [650]:
for gene in shared_genes:
    data_all[gene] = 0
data_all

,Class,Gene,ID,Variation,Text,Text_words,NOTCH1,MED12,RAD54L,CTNNB1,...,MET,PTEN,MYC,PIK3CA,BRAF,IDH2,XPO1,TP53,RAD50,SMAD4
0,2,CBL,1,W802*,Using select c-CBL somatic mutations such as S...,5783,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,CBL,2,Q249E,Using select c-CBL somatic mutations such as S...,5783,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,CBL,3,N454D,HRM analysis of CBL exons 8 and 9 in blast cri...,5625,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,CBL,4,L399V,Only V430M mutant was on the borderline with t...,6248,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,CBL,5,V391I,Only V430M mutant was on the borderline with t...,6248,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,CBL,6,V430M,Figure 4 Figure 4 Structures of wild type (gre...,6248,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,4,CBL,8,Y371H,We investigated the mechanism by which CBL- pl...,12009,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,4,CBL,9,C384R,We investigated the mechanism by which CBL-Y37...,5761,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,4,CBL,10,P395A,Only V430M mutant was on the borderline with t...,6248,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,4,CBL,11,K382E,"Purified PCR amplicons (CBLWT, CBLC381A, CBL p...",12251,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [651]:
for i in data_all.index:
    gene = data_all.Gene[i]
    if gene in shared_genes:
        data_all.loc[i, gene] = 1

data_all # it's not all zeros, it looks like it but genes like BRAF and EFGR have more 1's because they occur more

,Class,Gene,ID,Variation,Text,Text_words,NOTCH1,MED12,RAD54L,CTNNB1,...,MET,PTEN,MYC,PIK3CA,BRAF,IDH2,XPO1,TP53,RAD50,SMAD4
0,2,CBL,1,W802*,Using select c-CBL somatic mutations such as S...,5783,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,CBL,2,Q249E,Using select c-CBL somatic mutations such as S...,5783,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,CBL,3,N454D,HRM analysis of CBL exons 8 and 9 in blast cri...,5625,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,CBL,4,L399V,Only V430M mutant was on the borderline with t...,6248,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,CBL,5,V391I,Only V430M mutant was on the borderline with t...,6248,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,CBL,6,V430M,Figure 4 Figure 4 Structures of wild type (gre...,6248,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,4,CBL,8,Y371H,We investigated the mechanism by which CBL- pl...,12009,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,4,CBL,9,C384R,We investigated the mechanism by which CBL-Y37...,5761,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,4,CBL,10,P395A,Only V430M mutant was on the borderline with t...,6248,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,4,CBL,11,K382E,"Purified PCR amplicons (CBLWT, CBLC381A, CBL p...",12251,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [679]:
shared_genes_var = [gene + '_gene' for gene in shared_genes]

In [681]:
# Replace column name of genes so that they don't interfere with other processings
data_all.columns.values[6:] = shared_genes_var
data_all

,Class,Gene,ID,Variation,Text,Text_words,NOTCH1_gene,MED12_gene,RAD54L_gene,CTNNB1_gene,...,MET_gene,PTEN_gene,MYC_gene,PIK3CA_gene,BRAF_gene,IDH2_gene,XPO1_gene,TP53_gene,RAD50_gene,SMAD4_gene
0,2,CBL,1,W802*,Using select c-CBL somatic mutations such as S...,5783,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,CBL,2,Q249E,Using select c-CBL somatic mutations such as S...,5783,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,CBL,3,N454D,HRM analysis of CBL exons 8 and 9 in blast cri...,5625,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,CBL,4,L399V,Only V430M mutant was on the borderline with t...,6248,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,CBL,5,V391I,Only V430M mutant was on the borderline with t...,6248,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,CBL,6,V430M,Figure 4 Figure 4 Structures of wild type (gre...,6248,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,4,CBL,8,Y371H,We investigated the mechanism by which CBL- pl...,12009,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,4,CBL,9,C384R,We investigated the mechanism by which CBL-Y37...,5761,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,4,CBL,10,P395A,Only V430M mutant was on the borderline with t...,6248,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,4,CBL,11,K382E,"Purified PCR amplicons (CBLWT, CBLC381A, CBL p...",12251,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [666]:
new_train_1 = data_all.iloc[:len(new_train)]
new_test_1 = data_all.iloc[len(new_train):]

In [682]:
new_train_1.shape

(2925, 40)

In [683]:
new_test_1.shape

(619, 40)

In [66]:
new_train_1.to_csv("checkpoints_databases/w_working_train_new.csv",index=False,encoding="utf8")
new_test_1.to_csv("checkpoints_databases/w_working_test_new.csv",index=False,encoding="utf8")